<a href="https://colab.research.google.com/github/fefosalvati/Embedding-code/blob/main/EU_cose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏛️ EU Parliament Debates Search Interface

This notebook provides a user interface for searching through EU Parliament debates.

In [ ]:
!pip install ipywidgets pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import requests
from datetime import datetime

In [ ]:
# Replace this with your ngrok URL
API_URL = "https://64a8-185-8-197-4.ngrok-free.app/" # e.g., "https://your-ngrok-url.ngrok.io"

# Validate and format the API URL
if not API_URL:
    print("⚠️ Please set your ngrok URL in the API_URL variable above")
else:
    API_URL = API_URL.rstrip('/')

In [ ]:
def create_search_interface():
    # Create widgets
    query_input = widgets.Textarea(
        value='',
        placeholder='Enter your search query...',
        description='Query:',
        layout=widgets.Layout(
            width='70%',
            height='100px'  # Increased height for multiline
        ),
        continuous_update=False  # Only update on Enter/submission
    )

    limit_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=20,
        step=1,
        description='Limit:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

    search_button = widgets.Button(
        description='🔍 Search',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )

    output = widgets.Output()

    def format_result(result):
        """Format a single search result with HTML styling"""
        return f"""
        <div style="margin-bottom: 20px; padding: 15px; border: 1px solid #ddd; border-radius: 5px;">
            <div style="margin-bottom: 10px;">
                <span style="font-weight: bold;">{result['speaker_name']}</span>
                <span style="color: #666;"> • {result['speaker_party']} • {result['speaker_role']}</span>
            </div>
            <div style="margin-bottom: 10px; color: #fff;">
                <strong>Debate:</strong> {result['debate_title']}
            </div>
            <div style="margin-bottom: 10px; font-style: italic; color: #666;">
                {result['date']} (Year: {result['year']})
            </div>
            <div style="margin-bottom: 5px;">
                {result['text']}
            </div>
            <div style="color: #888; font-size: 0.9em;">
                Distance: {result['distance']:.3f}
            </div>
        </div>
        """

    def on_search_button_clicked(b):
        with output:
            clear_output()

            if not API_URL:
                print("⚠️ Please set the API_URL variable first!")
                return

            if not query_input.value.strip():
                print("⚠️ Please enter a search query")
                return

            try:
                # Show loading message
                print("🔄 Searching...")

                # Make API request
                response = requests.get(
                    f"{API_URL}/search/",
                    params={
                        'query': query_input.value,
                        'limit': limit_slider.value
                    }
                )

                response.raise_for_status()
                results = response.json()

                clear_output()

                if not results:
                    print("ℹ️ No results found")
                    return

                # Display results count
                display(HTML(f"<h4>📊 Found {len(results)} results</h4>"))

                # Display formatted results
                html_results = ''.join([format_result(result) for result in results])
                display(HTML(html_results))

            except requests.exceptions.ConnectionError:
                print("❌ Error: Could not connect to the API. Please check if the ngrok URL is correct and running.")
            except Exception as e:
                print(f"❌ Error: {str(e)}")

    search_button.on_click(on_search_button_clicked)

    # Layout
    input_box = widgets.VBox([
        widgets.HTML("<h3>Search EU Parliament Debates</h3>"),
        query_input,
        widgets.HBox([limit_slider, search_button],
                    layout=widgets.Layout(margin='10px 0'))
    ])

    display(input_box)
    display(output)

# Create the interface
create_search_interface()

Output()